In [2]:
import re
import pandas as pd
from openpyxl import load_workbook
pd.options.display.max_rows = None

In [3]:
# 開啟檔案
def open_brd_file(filepath):
    f = open(filepath)
    return f

# parse net name
def parse_net_name(string):
    # 定義正規表達式
    netNameRex = re.compile(r"^\s-?[A-Z]+[0-9]*_?[A-Z]+[0-9]*.*")
    netName = netNameRex.findall(string)
    return netName

# 找到總長度的欄位
def total(string):
    # 定義正規表達式
    totalRex = re.compile("TOTAL")
    total = totalRex.findall(string)
    return total
#找到位置
def parse_net(string):
#     locationRex = re.compile(r"([U|R|C|L|J]\d+.[A-Z]*\d*|VIA\(T\)|VIA\s)")
#     locationRex = re.compile(r"([U|R|C|L|J][A-Z]+\d+\.[A-Z]*\d*|[U|R|C|L|J][A-Z]+\.[A-Z]*\d*|[U|R|C|L|J]\d+.[A-Z]*\d*|[U|R|C|L|J][A-Z]*\d+.[A-Z]*\d*|VIA\(T\)|VIA\s)")
    locationRex = re.compile(r"([U][A-Z]+\d+\.[A-Z]*\d*|[U][A-Z]+\.[A-Z]*\d*|[U]\d+\.[A-Z]*\d*|[U][A-Z]*\d+\.[A-Z]*\d*|VIA\(T\)|VIA\s)")
    location = locationRex.findall(string)
    if location and location[0] == "VIA ":
        location[0] = location[0].replace(" ", "")
    return location

#找到單一段長度
def sub_length(string):
    lengthRex = re.compile(r"(\d+\.\d+)")
    length = lengthRex.findall(string)
    if length:
        length = length[-1]
    return length 

# 找到層數
def layer_num(string):
    layerRex = re.compile(r"BOTTOM|TOP|L\d+$|IN\d+$")
    layer = layerRex.findall(string)
    return layer

#找到總長度
def total_length(string):
    ttlLength = re.findall(r"\d+.\d+", string)[0]
    return ttlLength

#存csv
def save_csv(datalist, filename):
    df = pd.DataFrame(datalist)
    df.to_csv(f'{filename}.csv', index = False)
    
# 存excel
def save_excel(write, df, sheetName):
#     df = pd.DataFrame(datalist)
    df.to_excel(write, sheet_name=f'{sheetName}', index=False)
    
#刪多餘的raw
def deleteNullVIAROW(df):
    for i in range(df.shape[0]):
        if df.loc[i, "location"] == "VIA" and df.loc[i, "gap"] == 0.0:
            df.drop(i, axis=0, inplace=True)
            
#這一份txt total的path 數量
def branchPathNum(df):
    #扣掉net name/start_end/total length
    maxPathNum = int((df.shape[1] - 7) / 2)
    return maxPathNum

In [17]:
#step1: 開啟檔案
filepath = r"04291.txt"
f = open_brd_file(filepath)
print(f"step1: {filepath} open.")

#step2: 解析檔案
SQS = []
summary = []
netNameList = []

for i in f:
    i = i.replace("\n","")#去掉換行符號
    #如果不是net name 也不是 total
    if not parse_net_name(i):
        if not parse_net(i):
            if not total(i):
                continue
            
    #如果是net name
    if parse_net_name(i):
        data = {} #清空
        netName = i.replace(" ", "")
        #建立netnamelist
        netNameList.append(netName)
        netPath = ""
#         data.update({"net_name" : netName})
#         SQS.append(data)
#         print(data)
    
    #如果是location
#     elif parse_net(i):
#         data = {} #清空
        
#         if sub_length(i):
#             location = parse_net(i)[0]
#             length = float(sub_length(i))
#             layer = layer_num(i)[0] if layer_num(i) else ""
#             data.update({"net_name" : netName, "location" : location, "length": length, "layer": layer})
# #         print(data)
#             SQS.append(data)
    elif parse_net(i):
        data = {} #清空            
        if sub_length(i):
            location = parse_net(i)[0]
            
            length = float(sub_length(i))
            layer = layer_num(i)[0] if layer_num(i) else ""
            data.update({"net_name" : netName, "location" : location, "length": length, "layer": layer})
            SQS.append(data)
        
    
    #如果是total那一行
    elif total(i):
        data = {} #清空
        ttlLength = float(total_length(i))
        data.update({"net_name" : netName, "total_length" : ttlLength})
#         print(data)
        summary.append(data)
        
print(f"step2: {filepath} parsed done.")




step1: 04291.txt open.
step2: 04291.txt parsed done.


In [50]:
# 開啟檔案
def open_brd_file(filepath):
    f = open(filepath)
    return f

#用空格拆分每一行
def splitRow(string):
    string = string.split()
    return string

#刪除|開頭
def deleteRow1(string):
    pattern = re.compile(r"\|")
    delete = pattern.findall(string)
    return delete
#刪除- - -開頭
def deleteRow2(string):
    pattern = re.compile(r"^\-$")
    delete = pattern.findall(string)
    return delete

#刪除R開頭
def deleteRow3(string):
    pattern = re.compile(r"^R")
    delete = pattern.findall(string)
    return delete

In [51]:
#step1: 開啟檔案
filepath = r"04291.txt"
try:
    f = open_brd_file(filepath)
    print(f"step1: Open file. (File name: {filepath}) -DONE")
except:
    print(f"File is not exit, check file name.")
    
    
#step2: 刪除不必要行數
rawData = []
SQS = []
summary = []
netNameList = []

for i in f:
    i = i.replace("\n","")#去掉換行符號
    row = splitRow(i)

    if not deleteRow1(row[0]):
        if not deleteRow2(row[0]):
            rawData.append(row)
print(f"step2: Clear excess rows. -DONE")

#step3: 整理成需要的格式
for i in rawData:
#     if len(i) == 1: #netname那一行

    if len(i) < 3: #netname那一行 ######################### TODO這行應該要改
        data = {} #清空data
        netName = i[0]
        
        netNameList.append(netName) #建立netnamelist
    elif i[-1] == "0.000": #找到location

        data = {} #清空 
        try:
            location, length = i[0], float(i[-1])
            data.update({"net_name" : netName, "location" : location, "length": length, "layer": ""})
        except:
            print(netName," has error.")
        
        SQS.append(data)
    elif i[-1] == "mils": #找到TOTAL
        data = {} #清空
        ttlLength = float(i[3])
        data.update({"net_name" : netName, "total_length" : ttlLength})
        summary.append(data)
        
    else:
        data = {} #清空
        if not deleteRow3(i[0]):
            try:
                location, length, layer = i[0], float(i[-2]), i[-1]
                data.update({"net_name" : netName, "location" : location, "length": length, "layer": layer})
            except:
                print(netName, "has error.")

            SQS.append(data)

#存取df
dfsummary, dfSQS, dfSQSR = pd.DataFrame(summary), pd.DataFrame(SQS), pd.DataFrame(SQS)
print("step3: Transfer file format. -DONE")

step1: Open file. (File name: 04291.txt) -DONE
step2: Clear excess rows. -DONE
step3: Transfer file format. -DONE


In [40]:
SQS

[{'net_name': '-FBA_CLK0',
  'location': 'UGPU1.D25*',
  'length': 0.0,
  'layer': ''},
 {'net_name': '-FBA_CLK0',
  'location': 'VIA',
  'length': 25.236,
  'layer': 'BOTTOM'},
 {'net_name': '-FBA_CLK0',
  'location': 'VIA',
  'length': 1517.139,
  'layer': 'IN1'},
 {'net_name': '-FBA_CLK0',
  'location': 'UV1.K10*',
  'length': 1538.018,
  'layer': 'BOTTOM'},
 {'net_name': '-FBA_CLK1',
  'location': 'UGPU1.M22*',
  'length': 0.0,
  'layer': ''},
 {'net_name': '-FBA_CLK1',
  'location': 'VIA',
  'length': 22.46,
  'layer': 'BOTTOM'},
 {'net_name': '-FBA_CLK1',
  'location': 'VIA',
  'length': 2067.456,
  'layer': 'IN3'},
 {'net_name': '-FBA_CLK1',
  'location': 'UV2.K10*',
  'length': 2166.508,
  'layer': 'BOTTOM'},
 {'net_name': '-FBA_WCK01',
  'location': 'UGPU1.C18*',
  'length': 0.0,
  'layer': ''},
 {'net_name': '-FBA_WCK01',
  'location': 'VIA',
  'length': 22.129,
  'layer': 'BOTTOM'},
 {'net_name': '-FBA_WCK01',
  'location': 'VIA',
  'length': 1395.88,
  'layer': 'IN3'},
 {'n

In [24]:
len(SQS)

557

In [19]:
SQS1 = SQS
len(SQS)

552

In [52]:
# step3: 分析檔案 轉成sheet1 ans 2
#存取df
dfsummary, dfSQS, dfSQSR = pd.DataFrame(summary), pd.DataFrame(SQS), pd.DataFrame(SQS)

#修改df
for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
    
    # 填第一個元件的layer空格 
    # ex. TOP = TOP
    dfSQSR.loc[indexList[0],"layer"] = dfSQSR.loc[indexList[1],"layer"]
    
    #每個netname裡面找gap
    length = len(indexList)
    for index in indexList[::-1]:
        if length > 1:
            gap = dfSQSR.loc[index]["length"] - dfSQSR.loc[index-1]["length"]
            dfSQSR.loc[index,"gap"] = gap
        else:
            gap = 0
            dfSQSR.loc[index,"gap"] = gap
        length -= 1


#刪除多餘的VIA
deleteNullVIAROW(dfSQSR)

#在summary建立branch path & branch length
for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
    path = ""
    connIdxList = []
    MS = 0
    SL = 0
    NAN = 0
    
    for idx in indexList:
        
        ####################################################  前面的表層裡層         
        if re.findall("BOTTOM|TOP", dfSQSR.loc[idx, "layer"]): 
            MS += dfSQSR.loc[idx, "gap"]
        elif re.findall("L\d+$|IN\d+$", dfSQSR.loc[idx, "layer"]):
            SL += dfSQSR.loc[idx, "gap"]
        else:
            NAN += dfSQSR.loc[idx, "gap"]
        
        ##########################################################
        if not re.findall("VIA", dfSQSR.loc[idx, "location"]): #找出connnector index 排出VIA, VIA(T)
            connIdxList.append(idx)
    
    length = len(connIdxList)
    
    #找出connector 在每條indexlist 裡面的index
    num = 0
    netIndex = dfsummary[dfsummary["net_name"] == f"{i}"].index.tolist()
    
    #把起始點&終點 加入summary
    start_end = f"{dfSQSR.loc[indexList[0], 'location']}:{dfSQSR.loc[indexList[-1], 'location']}"

    dfsummary.loc[netIndex, "start_end_path"] = start_end
    
    ####################在這裡加入表層裡層
    dfsummary.loc[netIndex, "path_MS"] = start_end + "-MS"
    dfsummary.loc[netIndex, "length_MS"] = MS
    dfsummary.loc[netIndex, "path_SL"] = start_end + "-SL"
    dfsummary.loc[netIndex, "length_SL"] = SL
    
    
    for idx in range(length - 1):
        num +=1 
        pathIdx = []
        pathIdx.append(indexList.index(connIdxList[idx]))
        pathIdx.append(indexList.index(connIdxList[idx + 1]))
               
        #抓出兩兩conn
        connector = ""
        for j in pathIdx:
            connector += dfSQSR.loc[indexList[j], "location"] + ":"
        
        connector = connector[:-1] 
        dfsummary.loc[netIndex, f"path{num}"] = connector
        
        #算出兩兩conn間的長度
        branchLen = 0
        for k in indexList[ pathIdx[0]+1 : pathIdx[1]+1]:
            branchLen += dfSQSR.loc[k, "gap"]

        dfsummary.loc[netIndex, f"length{num}"] = branchLen

        
#調整dfsummary的順序
startEndColumn = dfsummary.pop(dfsummary.columns[2])
dfsummary.insert(1, startEndColumn.name, startEndColumn)
   

#轉成final SQS資料
column1 = []
column2 = []

#表格2的columns
column3 = []
column4 = []
column5 = []

for row in range(dfsummary.shape[0]):
    column1.append(dfsummary.loc[row,"net_name"])
#     column1.append(dfsummary.loc[row,"start_end_path"])
    column1.append(dfsummary.loc[row,"path_MS"])
    column1.append(dfsummary.loc[row,"path_SL"])
    
    column2.append("")
#     column2.append(dfsummary.loc[row,"total_length"])
    column2.append(dfsummary.loc[row,"length_MS"])
    column2.append(dfsummary.loc[row,"length_SL"])
    
    #append到表格2的
    column3.append(dfsummary.loc[row,"net_name"])
    column3.append(dfsummary.loc[row,"net_name"])
    
    column4.append(dfsummary.loc[row,"path_MS"])
    column4.append(dfsummary.loc[row,"path_SL"])
    
    column5.append(dfsummary.loc[row,"length_MS"])
    column5.append(dfsummary.loc[row,"length_SL"])


    for num in range(branchPathNum(dfsummary)):

        column1.append(dfsummary.loc[row,f"path{num + 1}"])
        column2.append(dfsummary.loc[row,f"length{num + 1}"])
        
final = {
    "SQS"    : column1,
    "length" : column2
}

final2 = {
    "netname" : column3,
    "SQS"     : column4,
    "length"  : column5
}
#轉成df 順便去掉空值
dfFinal = pd.DataFrame(final).dropna(axis=0)        
dfFinal2 = pd.DataFrame(final2).dropna(axis=0)   


print(f"step3: {filepath} converted to sheet1 and sheet2.")
  
    


step3: 04291.txt converted to sheet1 and sheet2.


In [53]:
#step4: 做成sheet3
#轉成final SQS資料
sheet3column1 = []
sheet3column2 = []
sheet3column3 = []

for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()

    for idx in indexList:
        location = dfSQSR.loc[idx, "location"]

        #整張表最後一個U 不然會報錯
        if idx == dfSQSR.shape[0] -1 and re.findall("^U", location):
#             u+=1
            #因為是最後一個U, 找他上一個是不是VIA
            if re.findall("^V", dfSQSR.loc[idx - 1, "location"]):
                preLocation = dfSQSR.loc[idx - 1, "location"]

                #path
                path = f"{location}:{preLocation}"
                #gap已經算好 
                gap = dfSQSR.loc[idx, "length"] - dfSQSR.loc[idx-1, "length"]
                
                sheet3column1.append(i)
                sheet3column2.append(path)
                sheet3column3.append(gap)

                
            #如果上一個不是VIA, 再往前找一個VIA    
            elif re.findall("^V", dfSQSR.loc[idx - 2, "location"]):
                pre2Location = dfSQSR.loc[idx - 2, "location"]
                
                #path
                path = f"{location}:{pre2Location}"
                
                #前一個gap
                preGap = dfSQSR.loc[idx - 1, "length"] - dfSQSR.loc[idx - 2, "length"]
                
                #gap ( 2nd U - VIA - (1st U -VIA)  )
                gap = dfSQSR.loc[idx, "length"] - dfSQSR.loc[idx - 2, "length"] - preGap
                
                sheet3column1.append(i)
                sheet3column2.append(path)
                sheet3column3.append(gap)

            
            
        #如果找到U
        elif re.findall("^U", location):
#             u+=1
            #下一步看他下一個是不是VIA
            #如果是就是U:VIA 的距離
            if re.findall("^V", dfSQSR.loc[idx + 1, "location"]):
                nextLocation = dfSQSR.loc[idx + 1, "location"]
                
                #path
                path = f"{location}:{nextLocation}"
                #gap
                gap = dfSQSR.loc[idx+1, "length"] - dfSQSR.loc[idx, "length"]
                
                sheet3column1.append(i)
                sheet3column2.append(path)
                sheet3column3.append(gap)

            
            #如果下一個不是VIA, 找他上一個是不是VIA
            elif re.findall("^V", dfSQSR.loc[idx - 1, "location"]):
                preLocation = dfSQSR.loc[idx - 1, "location"]

                #path
                path = f"{location}:{preLocation}"
                
                #gap已經算好 
                gap = dfSQSR.loc[idx, "length"] - dfSQSR.loc[idx-1, "length"]
                
                sheet3column1.append(i)
                sheet3column2.append(path)
                sheet3column3.append(gap)

                
            #如果下一個不是VIA, 上一個不是VIA, 再往前找一個    
            elif re.findall("^V", dfSQSR.loc[idx - 2, "location"]):
                pre2Location = dfSQSR.loc[idx - 2, "location"]
                
                #path
                path = f"{location}:{pre2Location}"
                
                #前一個gap
                preGap = dfSQSR.loc[idx - 1, "length"] - dfSQSR.loc[idx - 2, "length"]
                
                #gap ( 2nd U - VIA - (1st U -VIA)  )
                gap = dfSQSR.loc[idx, "length"] - dfSQSR.loc[idx - 2, "length"] - preGap
                
                sheet3column1.append(i)
                sheet3column2.append(path)
                sheet3column3.append(gap)

                
            else:
                print("not found")
                print(location)
                print(idx)
                print(dfSQSR.loc[idx - 2, "location"])
                print(re.findall("^U", location))
                
                print(i)
#         else:
#             print("False")
   
    
final3 = {
    "netname"  : sheet3column1,
    "path"     : sheet3column2,
    "length"   : sheet3column3
}

dfFinal3 = pd.DataFrame(final3).dropna(axis=0) 
    
print(f"step4: {filepath} converted sheet3 done.")    


step4: 04291.txt converted sheet3 done.


In [54]:
    
# step5: 儲存檔案
filename = re.findall(r"^\w*", filepath)[0]
write = pd.ExcelWriter(f'{filename}.xlsx')
save_excel(write, dfFinal, "final")
save_excel(write, dfFinal2, "final2")
save_excel(write, dfFinal3, "final3")
# save_excel(write, dfsummary, "summary")
# save_excel(write, dfSQS, "data base")
# save_excel(write, dfSQSR, "SQS")
write.save()

print(f"step5: {filename}.xlsx saved.")

step5: 04291.xlsx saved.


In [16]:
dfSQSR

,net_name,location,length,layer,gap
0,-FBA_CLK0,UGPU1.D25,0.000,BOTTOM,0.000
1,-FBA_CLK0,VIA,25.236,BOTTOM,25.236
2,-FBA_CLK0,VIA,1517.139,IN1,1491.903
3,-FBA_CLK0,UV1.K10,1538.018,BOTTOM,20.879
4,-FBA_CLK1,UGPU1.M22,0.000,BOTTOM,0.000
5,-FBA_CLK1,VIA,22.460,BOTTOM,22.460
6,-FBA_CLK1,VIA,2067.456,IN3,2044.996
7,-FBA_CLK1,UV2.K10,2166.508,BOTTOM,99.052
8,-FBA_WCK01,UGPU1.C18,0.000,BOTTOM,0.000
9,-FBA_WCK01,VIA,22.129,BOTTOM,22.129


In [83]:
dfSQSR.shape[0] -1

551

In [88]:
#轉成final SQS資料
sheet3column1 = []
sheet3column2 = []
sheet3column3 = []

for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
#     u=0
    for idx in indexList:
        location = dfSQSR.loc[idx, "location"]
        print(location)
        
        if idx == dfSQSR.shape[0] -1 and re.findall("^U", location):
#             u+=1
            #因為是最後一個U, 找他上一個是不是VIA
            if re.findall("^V", dfSQSR.loc[idx - 1, "location"]):
                preLocation = dfSQSR.loc[idx - 1, "location"]

                #path
                path = f"{location}:{preLocation}"
                #gap已經算好 
                gap = dfSQSR.loc[idx, "length"] - dfSQSR.loc[idx-1, "length"]
                
                sheet3column1.append(i)
                sheet3column2.append(path)
                sheet3column3.append(gap)
                
                print(f"{location}:{preLocation}")
                print(f"gap:{gap}")
                print(u)
                
            #如果上一個不是VIA, 再往前找一個VIA    
            elif re.findall("^V", dfSQSR.loc[idx - 2, "location"]):
                pre2Location = dfSQSR.loc[idx - 2, "location"]
                
                #path
                path = f"{location}:{pre2Location}"
                #gap
                gap = dfSQSR.loc[idx, "length"] - dfSQSR.loc[idx - 2, "length"]
                
                sheet3column1.append(i)
                sheet3column2.append(path)
                sheet3column3.append(gap)
                
                print(f"{location}:{pre2Location}")
                print(f"gap:{gap}")
                print(u)
            
            
        #如果找到U
        elif re.findall("^U", location):
#             u+=1
            #下一步看他下一個是不是VIA
            #如果是就是U:VIA 的距離
            if re.findall("^V", dfSQSR.loc[idx+1, "location"]):
                nextLocation = dfSQSR.loc[idx+1, "location"]
                
                #path
                path = f"{location}:{nextLocation}"
                #gap
                gap = dfSQSR.loc[idx+1, "length"] - dfSQSR.loc[idx, "length"]
                
                sheet3column1.append(i)
                sheet3column2.append(path)
                sheet3column3.append(gap)
                
                print(f"{location}:{nextLocation}")
                print(f"gap:{gap}")
                print(u)
            
            #如果下一個不是VIA, 找他上一個是不是VIA
            elif re.findall("^V", dfSQSR.loc[idx - 1, "location"]):
                preLocation = dfSQSR.loc[idx - 1, "location"]

                #path
                path = f"{location}:{preLocation}"
                
                #gap已經算好 
                gap = dfSQSR.loc[idx, "length"] - dfSQSR.loc[idx-1, "length"]
                
                sheet3column1.append(i)
                sheet3column2.append(path)
                sheet3column3.append(gap)
                print(f"{location}:{preLocation}")
                print(f"gap:{gap}")
                print(u)
                
            #如果下一個不是VIA, 上一個不是VIA, 再往前找一個    
            elif re.findall("^V", dfSQSR.loc[idx - 2, "location"]):
                pre2Location = dfSQSR.loc[idx - 2, "location"]
                
                #path
                path = f"{location}:{pre2Location}"
                #gap
                gap = dfSQSR.loc[idx, "length"] - dfSQSR.loc[idx - 2, "length"]
                
                
                sheet3column1.append(i)
                sheet3column2.append(path)
                sheet3column3.append(gap)
                
                print(f"{location}:{pre2Location}")
                print(f"gap:{gap}")
                print(u)
                
            else:
                print("not found")
                print(u)
        
        else:
            print("False")
        print("-"*80)
    print("="*80)
    
    
final3 = {
    "netname"  : sheet3column1,
    "path"     : sheet3column2,
    "length"   : sheet3column3
}

dfFinal3 = pd.DataFrame(final3).dropna(axis=0) 

UGPU1.D25
UGPU1.D25:VIA
gap:25.236
1
--------------------------------------------------------------------------------
VIA
False
--------------------------------------------------------------------------------
VIA
False
--------------------------------------------------------------------------------
UV1.K10
UV1.K10:VIA
gap:20.879000000000133
1
--------------------------------------------------------------------------------
UGPU1.M22
UGPU1.M22:VIA
gap:22.46
1
--------------------------------------------------------------------------------
VIA
False
--------------------------------------------------------------------------------
VIA
False
--------------------------------------------------------------------------------
UV2.K10
UV2.K10:VIA
gap:99.05199999999968
1
--------------------------------------------------------------------------------
UGPU1.C18
UGPU1.C18:VIA
gap:22.129
1
--------------------------------------------------------------------------------
VIA
False
----------------------

False
--------------------------------------------------------------------------------
VIA
False
--------------------------------------------------------------------------------
UGPU1.E13
UGPU1.E13:VIA
gap:21.759000000000015
1
--------------------------------------------------------------------------------
UGPU1.D13
UGPU1.D13:VIA
gap:24.331
1
--------------------------------------------------------------------------------
VIA
False
--------------------------------------------------------------------------------
VIA
False
--------------------------------------------------------------------------------
UV1.E12
UV1.E12:VIA
gap:20.878999999999905
1
--------------------------------------------------------------------------------
UGPU1.B15
UGPU1.B15:VIA
gap:23.977
1
--------------------------------------------------------------------------------
VIA
False
--------------------------------------------------------------------------------
VIA
False
-----------------------------------------------

gap:21.874
1
--------------------------------------------------------------------------------
VIA
False
--------------------------------------------------------------------------------
VIA
False
--------------------------------------------------------------------------------
UV1.C13
UV1.C13:VIA
gap:20.878999999999905
1
--------------------------------------------------------------------------------
UV1.T13
UV1.T13:VIA
gap:20.879
1
--------------------------------------------------------------------------------
VIA
False
--------------------------------------------------------------------------------
VIA
False
--------------------------------------------------------------------------------
UGPU1.B16
UGPU1.B16:VIA
gap:22.69100000000003
1
--------------------------------------------------------------------------------
UGPU1.B22
UGPU1.B22:VIA
gap:22.461
1
--------------------------------------------------------------------------------
VIA
False
---------------------------------------------

In [89]:
dfFinal3

,netname,path,length
0,-FBA_CLK0,UGPU1.D25:VIA,25.236
1,-FBA_CLK0,UV1.K10:VIA,20.879
2,-FBA_CLK1,UGPU1.M22:VIA,22.460
3,-FBA_CLK1,UV2.K10:VIA,99.052
4,-FBA_WCK01,UGPU1.C18:VIA,22.129
5,-FBA_WCK01,UV1.D5:VIA,64.283
6,-FBA_WCK23,UGPU1.D16:VIA,23.781
7,-FBA_WCK23,UV1.R10:VIA,21.498
8,-FBA_WCK45,UGPU1.U24:VIA,24.329
9,-FBA_WCK45,UV2.D5:VIA,20.879


In [46]:
for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
    netPath = []
    for idx in indexList:
        netPath.append(dfSQSR.loc[idx, "location"])
    print(indexList)
    print(netPath)
    u = 0
    v =0
    for location in netPath:
        print(location)
        
        if re.findall("U", location):
            print(re.findall("^U", location))
            u += 1
            print("find U:  ", location)
            print("u quanty: ", u)
    print("~~~~~~~~~~~~~~")    
    print(indexList)        
#     if u == 2:
#         dfSQSR.loc[idx, "location"]

    print("-"*100)       

[0, 1, 2, 3]
['UGPU1.D25', 'VIA', 'VIA', 'UV1.K10']
UGPU1.D25
['U']
find U:   UGPU1.D25
u quanty:  1
VIA
VIA
UV1.K10
['U']
find U:   UV1.K10
u quanty:  2
~~~~~~~~~~~~~~
[0, 1, 2, 3]
----------------------------------------------------------------------------------------------------
[4, 5, 6, 7]
['UGPU1.M22', 'VIA', 'VIA', 'UV2.K10']
UGPU1.M22
['U']
find U:   UGPU1.M22
u quanty:  1
VIA
VIA
UV2.K10
['U']
find U:   UV2.K10
u quanty:  2
~~~~~~~~~~~~~~
[4, 5, 6, 7]
----------------------------------------------------------------------------------------------------
[8, 9, 10, 11]
['UGPU1.C18', 'VIA', 'VIA', 'UV1.D5']
UGPU1.C18
['U']
find U:   UGPU1.C18
u quanty:  1
VIA
VIA
UV1.D5
['U']
find U:   UV1.D5
u quanty:  2
~~~~~~~~~~~~~~
[8, 9, 10, 11]
----------------------------------------------------------------------------------------------------
[12, 13, 14, 15]
['UGPU1.D16', 'VIA', 'VIA', 'UV1.R10']
UGPU1.D16
['U']
find U:   UGPU1.D16
u quanty:  1
VIA
VIA
UV1.R10
['U']
find U:   UV1.R10
u qua

UGPU1.J27
['U']
find U:   UGPU1.J27
u quanty:  1
VIA
VIA(T)
UV2.K12
['U']
find U:   UV2.K12
u quanty:  2
UV2.J12
['U']
find U:   UV2.J12
u quanty:  3
~~~~~~~~~~~~~~
[182, 183, 184, 185, 186]
----------------------------------------------------------------------------------------------------
[187, 188, 189, 190, 191]
['UGPU1.J26', 'VIA', 'VIA(T)', 'UV2.K11', 'UV2.J11']
UGPU1.J26
['U']
find U:   UGPU1.J26
u quanty:  1
VIA
VIA(T)
UV2.K11
['U']
find U:   UV2.K11
u quanty:  2
UV2.J11
['U']
find U:   UV2.J11
u quanty:  3
~~~~~~~~~~~~~~
[187, 188, 189, 190, 191]
----------------------------------------------------------------------------------------------------
[192, 193, 194, 195]
['UGPU1.B19', 'VIA', 'VIA', 'UV1.H12']
UGPU1.B19
['U']
find U:   UGPU1.B19
u quanty:  1
VIA
VIA
UV1.H12
['U']
find U:   UV1.H12
u quanty:  2
~~~~~~~~~~~~~~
[192, 193, 194, 195]
----------------------------------------------------------------------------------------------------
[196, 197, 198, 199]
['UGPU1.F22', 'VI

VIA
UV2.U3
['U']
find U:   UV2.U3
u quanty:  2
~~~~~~~~~~~~~~
[424, 425, 426, 427]
----------------------------------------------------------------------------------------------------
[428, 429, 430, 431]
['UGPU1.T25', 'VIA', 'VIA', 'UV2.V3']
UGPU1.T25
['U']
find U:   UGPU1.T25
u quanty:  1
VIA
VIA
UV2.V3
['U']
find U:   UV2.V3
u quanty:  2
~~~~~~~~~~~~~~
[428, 429, 430, 431]
----------------------------------------------------------------------------------------------------
[432, 433, 434, 435]
['UGPU1.N27', 'VIA', 'VIA', 'UV2.U2']
UGPU1.N27
['U']
find U:   UGPU1.N27
u quanty:  1
VIA
VIA
UV2.U2
['U']
find U:   UV2.U2
u quanty:  2
~~~~~~~~~~~~~~
[432, 433, 434, 435]
----------------------------------------------------------------------------------------------------
[436, 437, 438, 439]
['UGPU1.R27', 'VIA', 'VIA', 'UV2.U4']
UGPU1.R27
['U']
find U:   UGPU1.R27
u quanty:  1
VIA
VIA
UV2.U4
['U']
find U:   UV2.U4
u quanty:  2
~~~~~~~~~~~~~~
[436, 437, 438, 439]
-----------------------------